In [13]:
import numpy as np
import scipy
import os
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten
from keras.layers import Conv2D, MaxPooling2D
from keras.optimizers import SGD
from keras import backend as K
K.set_image_data_format('channels_last')
from keras.utils import np_utils
#from sklearn.cross_validation import StratifiedKFold

In [14]:
PATH = os.getcwd()
print (PATH)

C:\Users\win10\Downloads\Classification-of-Hyperspectral-Image-master


In [15]:
# Load the Global values (windowSize, numPCAcomponents, testRatio) from the text file global_variables.txt
myFile = open('global_variables.txt', 'r') 
file = myFile.readlines()[:]


for line in file:

    if line[0:3] == "win":

        ds = line.find('=')
        windowSize = int(line[ds+1:-1],10)

    elif line[0:3] == "num":

        ds = line.find('=')
        numPCAcomponents = int(line[ds+2:-1],10)

    else:

        ds = line.find('=')
        testRatio = float(line[ds+1:])


In [16]:
# Global Variables
#windowSize = 5
#numPCAcomponents = 30
#testRatio = 0.25

# Load Training Dataset

In [17]:
X_train = np.load("X_trainPatches_" + str(windowSize) + "PCA" + str(numPCAcomponents) + "testRatio" + str(testRatio)  + ".npy")

y_train = np.load("y_trainPatches_" + str(windowSize) + "PCA" + str(numPCAcomponents) + "testRatio" + str(testRatio) + ".npy")

In [18]:
# Reshape into (numberofsamples, channels, height, width)
X_train = np.reshape(X_train, (X_train.shape[0],X_train.shape[3], X_train.shape[1], X_train.shape[2]))

In [19]:
# convert class labels to on-hot encoding
y_train = np_utils.to_categorical(y_train)

In [20]:
# Define the input shape 
input_shape= X_train[0].shape
print(input_shape)

(30, 5, 5)


In [21]:
# number of filters
C1 = 3*numPCAcomponents

In [22]:
# Define the model
model = Sequential()

model.add(Conv2D(C1, (3, 3), activation='relu', input_shape=input_shape))
model.add(Conv2D(3*C1, (3, 3), activation='relu'))
model.add(Dropout(0.25))



model.add(Flatten())
model.add(Dense(6*numPCAcomponents, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(16, activation='softmax'))

In [23]:
sgd = SGD(lr=0.0001, decay=1e-6, momentum=0.9, nesterov=True)
model.compile(loss='categorical_crossentropy', optimizer=sgd, metrics=['accuracy'])

In [24]:
model.fit(X_train, y_train, batch_size=32, epochs=15)

Epoch 1/15
928/928 [==============================] - 51s 54ms/step - loss: 2.4491 - accuracy: 0.2856
Epoch 2/15
928/928 [==============================] - 54s 58ms/step - loss: 1.6286 - accuracy: 0.5566
Epoch 3/15
928/928 [==============================] - 50s 54ms/step - loss: 1.0846 - accuracy: 0.6724
Epoch 4/15
928/928 [==============================] - 46s 50ms/step - loss: 0.8301 - accuracy: 0.7388
Epoch 5/15
928/928 [==============================] - 46s 50ms/step - loss: 0.6899 - accuracy: 0.7754
Epoch 6/15
928/928 [==============================] - 46s 50ms/step - loss: 0.6044 - accuracy: 0.7985
Epoch 7/15
928/928 [==============================] - 46s 50ms/step - loss: 0.5420 - accuracy: 0.8216
Epoch 8/15
928/928 [==============================] - 48s 52ms/step - loss: 0.4988 - accuracy: 0.8343
Epoch 9/15
928/928 [==============================] - 48s 51ms/step - loss: 0.4590 - accuracy: 0.8469
Epoch 10/15
928/928 [==============================] - 47s 50ms/step - loss: 0.429

In [25]:
import h5py
from keras.models import load_model

In [26]:
model.save('new_my_model' + str(windowSize) + 'PCA' + str(numPCAcomponents) + "testRatio" + str(testRatio) + '.h5')